In [469]:
import pandas as pd
sample_submission = pd.read_csv('/Users/mattmastin/Desktop/Tanzania/sample_submission.csv')
train_features = pd.read_csv('/Users/mattmastin/Desktop/Tanzania/train_features.csv')
test_features = pd.read_csv('/Users/mattmastin/Desktop/Tanzania/test_features.csv')
train_labels = pd.read_csv('/Users/mattmastin/Desktop/Tanzania/train_labels.csv')

sample_submission.shape, train_features.shape, test_features.shape, train_labels.shape

((14358, 2), (59400, 40), (14358, 40), (59400, 2))

In [470]:
import category_encoders as ce
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import MissingIndicator
pd.set_option('display.float_format', '{:.2f}'.format)

In [471]:
train = pd.merge(train_features, train_labels)

In [472]:
train, val = train_test_split(train, train_size=0.80, test_size=0.20,
                             stratify=train['status_group'], random_state=42)

In [473]:
train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 39))

In [474]:
train.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    2904
gps_height                   0
installer                 2917
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 286
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            2644
recorded_by                  0
scheme_management         3128
scheme_name              22532
permit                    2443
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [475]:
train.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,47520.00,47520.00,47520.00,47520.00,47520.00,47520.00,47520.00,47520.00,47520.00,47520.00
mean,37037.92,321.93,669.57,34.08,-5.71,0.48,15.26,5.62,179.81,1302.77
std,21412.10,3197.24,693.01,6.55,2.94,13.31,17.53,9.62,463.08,950.96
min,0.00,0.00,-63.00,0.00,-11.65,0.00,1.00,0.00,0.00,0.00
25%,18482.75,0.00,0.00,33.09,-8.53,0.00,5.00,2.00,0.00,0.00
50%,36986.50,0.00,372.50,34.91,-5.02,0.00,12.00,3.00,25.00,1986.00
75%,55450.25,25.00,1320.00,37.17,-3.33,0.00,17.00,5.00,215.00,2004.00
max,74247.00,350000.00,2770.00,40.34,-0.00,1776.00,99.00,80.00,15300.00,2013.00


In [482]:
train.describe(exclude='number')

,funder,installer,wpt_name,basin,subvillage,region,region_code,district_code,lga,ward,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
count,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
unique,1717,1930,30661,9,17232,21,27,20,124,2082,...,8,6,5,5,10,7,3,7,6,3
top,Government Of Tanzania,DWE,none,Lake Victoria,Shuleni,Iringa,11,1,Njombe,Igosi,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
freq,7321,13978,2879,8137,420,4250,4255,9772,2003,257,...,40598,40598,26567,26567,13620,13620,36638,22778,27642,25807


In [476]:
def wrangle(X):
    """Wrangles train, validate, and test sets in the same way"""
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    X['longitude'] = X['latitude'].replace(0, 34)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace them with the column mean.
    cols_with_zeros = ['construction_year', 'longitude', 'latitude']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col] = X[col].fillna(X[col].mean())
        
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract year from date_recorded
    X['year_recorded'] = X['date_recorded'].dt.year
    
    # quantity & quantity_group are duplicates, so drop one
#     X = X.drop(columns={'quantity_group', 'scheme_name', 'permit', 'public_meeting',
#               'scheme_management', 'subvillage', 'installer', 'funder'})
    
    # for categoricals with missing values, fill with the category 'MISSING'
    categoricals = X.select_dtypes(exclude='number').columns
#     for col in categoricals:
#         X[col] = X[col].fillna('MISSING')
    
    return X

In [477]:
def wrangle1(df):
    df = df.copy()
    
    df['latitude'] = df['latitude'].replace(-2e-08, 0)
    
    cols_with_zeros = ['construction_year', 'longitude', 'latitude']
    for col in cols_with_zeros:
        df[col] = df[col].replace(0, np.nan)
        df[col] = df[col].fillna(df[col].mean())
        
    df['date_recorded'] = pd.to_datetime(df['date_recorded'], 
                                         infer_datetime_format=True)
    
    df['year_recorded'] = df['date_recorded'].dt.year
    
#     df = df.drop(columns='quantity_group')
    
    categoricals = df.select_dtypes(exclude='number').columns
    for col in categoricals:
        df[col] = df[col].fillna('other')
        
    df['region_code'] = df['region_code'].astype('str')
    df['district_code'] = df['district_code'].astype('str')
    
    df = df.drop(columns='recorded_by')
    df = df.drop(columns='date_recorded')
    df = df.drop(columns='date_recorded')
    
    type_dict = {'amount_tsh':'float64',
                'gps_height':'float64',
                'longitude':'float64',
                'latitude':'float64',
                'num_private':'float64',
                'population':'float64',
                'construction_year':'float64'}
    df = df.astype(dtype = type_dict)
    
#     numeric = ['amount_tsh',
#               'gps_height',
#               'longitude',
#               'population',
#               'construction_year']
#     null_values = {'amount_tsh':0,
#               'gps_height':0,
#               'longitude':0,
#               'population':0,
#               'construction_year':0}
    
    df['population'] = df['population'].replace(0, 180)
    df['population'] = df['population'].replace(0, 180)
    df['construction_year'] = [year if 1960 < year < 2019 else 
                               1996 for year in df['construction_year']]
    
#     indicator = MissingIndicator()
#     trash_array = indicator.fit_transform(df[numeric])
    
#     trash_names = [numeric[x] + '_trash' for x in indicator.features_]
    
#     for feature in numeric:
#         replacements = df.groupby('ward')[feature].transform('mean')
#         df[feature] = df[feature].fillna(replacements)
        
#     for feature in numeric:
#         replacements = df.groupby('region')[feature].transform('mean')
#         df[feature] = df[feature].fillna(replacements)
        
#     for feature in numeric:
#         replacements = df[feature].median()
#         df[feature] = df[feature].fillna(replacements)
        
    return df

In [478]:
# def wrangle2(df):
#     df = df.copy()
    
# #     df = df.drop(columns='date_recorded')
    
#     categoricals = df.select_dtypes(exclude='number').columns.tolist()
    
# #     df[categoricals] = df[categoricals].applymap(lambda x: x.lower())
    
#     nan_list = ['not known', 'unknown', 'none', '-', '##', 'not kno', 'unknown installer']
#     df = df.replace(nan_list, np.nan)
    
#     for feature in df[categoricals]:
#         to_keep = df[feature].value_counts()[df[feature].value_counts() > 100].index.tolist()
#         feature_copy = df[feature].copy()
#         feature_copy[~feature_copy.isin(to_keep)] = np.nan
#         df[feature] = feature_copy
        
#     df[categoricals] = df[categoricals].fillna('other')
    
#     return df

In [479]:
train = wrangle(train)
val = wrangle(val)
test = wrangle(test_features)

In [480]:
train = wrangle1(train)
val = wrangle1(val)
test = wrangle1(test)

In [428]:
# train = wrangle2(train)
# val = wrangle2(val)
# test = wrangle2(test)

In [429]:
target = 'status_group'
train_features = train.drop(columns=[target, 'id'])
numeric_features = train_features.select_dtypes(include='number').columns.tolist()
cardinality = train_features.select_dtypes(exclude='number').nunique()
categorical_features = cardinality[cardinality <= 150].index.tolist()
features = numeric_features + categorical_features

In [430]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

In [431]:
# train_location = X_train[['longitude', 'latitude', 'amount_tsh', 'gps_height', 'num_private',
#                          'district_code', 'population', 'construction_year', 'year_recorded']].copy()
# val_location = X_val[['longitude', 'latitude', 'amount_tsh', 'gps_height', 'num_private',
#                          'district_code', 'population', 'construction_year', 'year_recorded']].copy()

In [432]:
# # trying to fix 'allow_nan=force_all_finite == 'allow-nan''error

# X_train_encoded = X_train_encoded.reset_index()
# y_train = y_train.reset_index()

In [433]:
X_train_encoded.shape, y_train.shape

((41580, 378), (41580,))

In [573]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=18, min_samples_split=3, presort=True)
dt.fit(X_train_encoded, y_train)
print('Decision Tree')
print('Train accuracy', dt.score(X_train_encoded, y_train))
print('Validation accuracy', dt.score(X_val_encoded, y_val))

Decision Tree
Train accuracy 0.8581529581529581
Validation accuracy 0.7704826038159371


In [571]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=33, min_samples_split=12,
                            criterion='gini', min_samples_leaf=4)
clf.fit(X_train_encoded, y_train)

/Users/mattmastin/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=33, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=12,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [572]:
clf.score(X_val_encoded, y_val)

0.7895061728395062

In [531]:
y_pred = clf.predict(X_test_encoded)
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-08.csv', index=False)

#### X_train.dtypes

In [95]:
X_train['quantity'].describe

<bound method NDFrame.describe of 43360    insufficient
7263           enough
2486     insufficient
313            enough
52726          enough
8558     insufficient
2559     insufficient
54735          enough
25763          enough
44540          enough
28603             dry
4372     insufficient
30666    insufficient
6431           enough
57420    insufficient
1373           enough
2026           enough
58977             dry
41101        seasonal
10019          enough
5103           enough
36712    insufficient
29670    insufficient
54588        seasonal
31867    insufficient
12090        seasonal
48300          enough
16754          enough
50582             dry
9136           enough
             ...     
26450          enough
56373    insufficient
4595     insufficient
35771          enough
41564          enough
30571    insufficient
25961    insufficient
21280    insufficient
40122          enough
42270          enough
38160          enough
6050           enough
12578          enoug